In [1]:
# Dependencies
import sys
import time
import json
import requests
import importlib
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
from config import gkey, zkey, zws_id, akey

In [2]:
# get list of zipcodes
%run ./write_zips.ipynb
import zipcodes
importlib.reload(zipcodes)
zip_codes = zipcodes.ziplist

In [3]:
# global variables
loading = '.'

In [4]:
# get http request headers
def getHeaders():
    headers = {
        "apikey":akey,
        "accept":"application/json"
    }
    return headers


# GET /property/address via zip code
def fetch(url, params):
    headers = getHeaders()
    res = requests.get(url, headers=headers, params=params).json()
    return res


# show load bar and wait for next request
def wait(t):
    sys.stdout.write(loading)
    sys.stdout.flush()
    time.sleep(t)

In [5]:
# get properties for each zip code (10 queries/min)
url = "https://search.onboard-apis.com/propertyapi/v1.0.0/property/address?"
sys.stdout.write('getting properties..')
property_results = []

for z in zip_codes:
    params = {
        "postalcode": z,
        "propertytype":"RESIDENTIAL (NEC)",
        "pagesize":"100"
    }
    
    res = fetch(url, params)
    
    property_results.append(res)
    wait(10)

getting properties........................................

In [118]:
# get attomIds from property results
properties = []

for p in property_results:
    for q in p['property']:
        properties.append(q)

props_df = pd.DataFrame(properties)
attomId_series = props_df['identifier']

attomIds = []
for a in attomId_df:
    attomIds.append(a['attomId'])

In [29]:
# get home values for each property (10 queries/min)
url = "https://search.onboard-apis.com/propertyapi/v1.0.0/property/basicprofile?"
sys.stdout.write("getting home values..")
hv_results = []

for a in attomIds:
    params = {
        "attomId": a,
        "page":"1",
        "pagesize":"100"
    }

    idres = fetch(url, params)
    hv_results.append(idres)    
    wait(10)

getting home values......................................................................................

In [327]:
prop_res = pd.DataFrame(hv_results)
props = prop_res['property']
zmkv = []

for i in range(len(props)):
    zmkv.append({"zipcode":props[i][0]['address']['postal1'], "mkt_val":props[i][0]['assessment']['market']['mktTtlValue']})

zip_mktval_df = pd.DataFrame(zmkv).set_index('zipcode')

mkt_sum = zip_mktval_df.groupby('zipcode').sum()/len(props)
zc_mkt_df = pd.DataFrame(mkt_sum)
zc_mkt_df = zc_mkt_df.reset_index()
    
# zc_mkt_df = zc_mkt_df.style.format({'mkt_val':'${:,.2f}'})
# for i in zc_mkt_df
zc_mkt_df

,zipcode,mkt_val
0,30303,224485.714286
1,30312,785823.809524


In [51]:
# pprint(hv_results[0]['property'][0]['address']['postal1'])
# pprint(hv_results[0]['property'][0]['assessment']['market']['mktTtlValue'])

In [ ]:
# cont.
# mkt_values = []

# for a in aid_results:
#     for p in a['property']:
#         mkt_values.append(p['assessment']['market']['mktTtlValue'])

# mkt_values